# ORIE 4330/5330 Project Part 1

This notebook is meant to get you started on part 1 of your term project. It includes code to read all of the input files and later on defines functions which you need to complete in order to model and solve the given IP.

In [43]:
# imports the modules we use throughout the notebook
import numpy as np
import pandas as pd
from gurobipy import *

## Load Dataframes

In this section we read all of the input data, and in some cases make edits that will help later on while building the model. Each cell contains comments explaining what file is being read or what the code is changing.

In [44]:
# path for input data
input_data_path = 'C:\\Users\\emily\\OneDrive\\Documents\\4330 Discrete Models\\Term Project\\'

In [45]:
# df with prelim exams requested
exams = (pd.read_csv(f'{input_data_path}prelim_exams.csv'))
exams

,exam_id,course,acadorg,enrollment,modality,prefdate,prefdate2,prefdate3
0,1-AEM-2210-LEC-2634167-1,AEM 2210,AEM,260,Online,2021-03-18,2021-03-16,2021-03-23
1,1-AEM-2210-LEC-2634167-2,AEM 2210,AEM,260,Online,2021-04-15,2021-04-13,2021-04-20
2,1-AEM-2225-LEC-2634167-1,AEM 2225,AEM,50,Online,2021-03-23,2021-03-18,2021-03-25
3,1-AEM-2225-LEC-2634167-2,AEM 2225,AEM,50,Online,2021-04-20,2021-04-15,2021-04-22
4,1-AEM-2240-LEC-3778494-1,AEM 2240,AEM,270,In person,2021-03-16,2021-03-04,2021-03-18
...,...,...,...,...,...,...,...,...
225,1-STSCI-1380-LEC-1757307-1,STSCI 1380,STSCI,64,In person,2021-03-16,2021-03-18,2021-03-04
226,1-STSCI-1380-LEC-1757307-2,STSCI 1380,STSCI,64,In person,2021-04-20,2021-04-22,2021-04-15
227,1-STSCI-2150-LEC-1319792-1,STSCI 2150,STSCI,140,In person,2021-03-18,2021-03-16,2021-03-23
228,1-STSCI-2150-LEC-1319792-2,STSCI 2150,STSCI,140,Online,2021-04-08,2021-04-06,2021-04-13


In [46]:
# since online exams do not a need a physical room, we can set the enrollment to 0
exams.loc[exams.modality == 'Online','enrollment'] = 0
exams

,exam_id,course,acadorg,enrollment,modality,prefdate,prefdate2,prefdate3
0,1-AEM-2210-LEC-2634167-1,AEM 2210,AEM,0,Online,2021-03-18,2021-03-16,2021-03-23
1,1-AEM-2210-LEC-2634167-2,AEM 2210,AEM,0,Online,2021-04-15,2021-04-13,2021-04-20
2,1-AEM-2225-LEC-2634167-1,AEM 2225,AEM,0,Online,2021-03-23,2021-03-18,2021-03-25
3,1-AEM-2225-LEC-2634167-2,AEM 2225,AEM,0,Online,2021-04-20,2021-04-15,2021-04-22
4,1-AEM-2240-LEC-3778494-1,AEM 2240,AEM,270,In person,2021-03-16,2021-03-04,2021-03-18
...,...,...,...,...,...,...,...,...
225,1-STSCI-1380-LEC-1757307-1,STSCI 1380,STSCI,64,In person,2021-03-16,2021-03-18,2021-03-04
226,1-STSCI-1380-LEC-1757307-2,STSCI 1380,STSCI,64,In person,2021-04-20,2021-04-22,2021-04-15
227,1-STSCI-2150-LEC-1319792-1,STSCI 2150,STSCI,140,In person,2021-03-18,2021-03-16,2021-03-23
228,1-STSCI-2150-LEC-1319792-2,STSCI 2150,STSCI,0,Online,2021-04-08,2021-04-06,2021-04-13


In [47]:
# read the room buckets
room_buckets = (pd.read_csv(f'{input_data_path}room_buckets.csv')
             .reset_index().set_index('bucket_size').to_dict()['num_rooms'])
room_buckets

{10: 7, 20: 41, 30: 20, 50: 13, 80: 8, 130: 1}

In [48]:
# adding the dummy room bucket to the room bucket dictionary
# since online exams now have enrollment set to 0, the dummy room bucket can have size 0
# and number of rooms the total number of exams
room_buckets[0] = len(exams)
room_buckets

{10: 7, 20: 41, 30: 20, 50: 13, 80: 8, 130: 1, 0: 230}

In [49]:
# reads the df with available exam dates and create a date -> date_index dictionary
# the set of date_index will be your set of days D on which you can schedule exams. 
exam_dates = (pd.read_csv(f'{input_data_path}avail_prel_dates.csv')
             .reset_index().set_index('exam_dates').to_dict()['index'])
# number of slots per day
K = 2
exam_dates

{'2021-02-25': 0,
 '2021-03-02': 1,
 '2021-03-04': 2,
 '2021-03-16': 3,
 '2021-03-18': 4,
 '2021-03-23': 5,
 '2021-03-25': 6,
 '2021-03-30': 7,
 '2021-04-01': 8,
 '2021-04-06': 9,
 '2021-04-08': 10,
 '2021-04-13': 11,
 '2021-04-15': 12,
 '2021-04-20': 13,
 '2021-04-22': 14,
 '2021-04-29': 15,
 '2021-05-04': 16,
 '2021-05-06': 17,
 '2021-05-11': 18,
 '2021-05-13': 19}

In [50]:
# reads the cornellment matrix
coenroll_s21 = (pd.read_csv(f'{input_data_path}coenrollment_s21.csv', index_col=0))
# coenrollment cutoff value kappa 
kappa = 1
# substract cutoff from all entries so they are in the form required for the obj fn
coenroll_s21 -= kappa
# these entries were not greater than kappa so can not have coenrollment conflicts
coenroll_s21[coenroll_s21 <= 0 ] = 0
coenroll_s21

,AEM 2210,AEM 2225,AEM 2240,AEM 2241,AEM 2300,AEM 2770,AEM 3100,AEM 3230,AEM 3370,AEM 4280,...,PHYS 2213,PHYS 2214,PHYS 2217,PHYS 2218,PHYS 3318,PHYS 4443,PSYCH 3420,STSCI 1380,STSCI 2150,BIOMI 2900
AEM 2210,242,0,0,4,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,2,5
AEM 2225,0,48,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AEM 2240,0,1,263,0,50,7,25,173,3,3,...,0,0,0,0,0,0,0,0,0,0
AEM 2241,4,0,0,192,0,0,0,4,0,0,...,0,0,0,0,0,0,2,0,0,2
AEM 2300,0,0,50,0,170,7,8,39,0,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PHYS 4443,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,9,37,0,0,0,0
PSYCH 3420,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,86,0,0,0
STSCI 1380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,0,0
STSCI 2150,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,214,7


## Building the Model

In this section we define a class that contains functions that are meant to implement the modeling process. By completing all of them, you will have finished creating the IP model for the prelim scheduling problem.
<br/>

You do not have to use the given template if you prefer not to, just make your code correctly models the problem and produces a feasible exam to day/slot assignment.

In [78]:
class PrelimExamScheduler():
    
    def __init__(self, 
                exams, 
                exam_dates,
                room_buckets,
                coenroll,
                K,
                R = 10,
                rooms_used_weight = 1,
                weight_pref_2 = 1,
                weight_pref_3 = 2):
        '''
        Inputs:
        - exams: dataframe with each prelim that must be scheduled. Has the following fields:
            - exam_id: the unique id for this prelim
            - enrollment: the enrollment for this exam 
            - modality: Online/In Pesron exam
            - course: course name
            - prefdate, prefdate2, prefdate3: prefered days for the exam
        - exam_dates: dictionary of date -> date_index
        - room_buckets: dictionary of room bucket size to number of rooms
        - coenroll: a dataframe whose indices and columns create the coenrollment conflict matrix 
        - K: the number of slots in a day
        - R: the maximum number of rooms that a prelim can use
        - rooms_used_weight,  weight_pref_2, weight_pref_3 
            are the weights of the corresponding terms in the objective function
        '''

        # save parameters
        self.exam_id = exams["exam_id"]
        self.exams = exams
        self.exam_dates = exam_dates
        self.room_buckets = room_buckets
        self.K = K
        self.R = R
        self.rooms_used_weight = rooms_used_weight
        self.weight_pref_2 = weight_pref_2
        self.weight_pref_3 = weight_pref_3
        # total number of exams to be scheduled
        self.M = len(self.exams)
        # fill out anything else you might need in the initialization
        self.coenroll_s21 = coenroll_s21

        return 

    def build_model(self):
        '''
        Function that creates the IP model
        '''
        self.model = Model("IP")
        # define gap between the solution found and the optimal bound
        self.model.setParam('MIPGap', .05)
        # initialize the decision variables 
        self.init_dv()
        # add constraints
        self.add_constraints()
        # set objective
        self.set_objective()

    def init_dv(self):
        '''
        Defines the decision variable
        '''
        # Define y(i,d,k) indicating if prelim i is assigned to day d and slot k
        t = []
        for i in exams.index:
            for k in range(1, K+1):
                t.append((i,exams.prefdate[i],k))
                t.append((i,exams.prefdate2[i],k))
                t.append((i,exams.prefdate3[i],k))
        self.y = self.model.addVars(t)

        # Define x(i,j,d,k) indicating the number of rooms of room_bucket j 
        # prelim i is using on day d and slot k
        u = []
        for i in exams.index:
            for d in exam_dates:
                for r in room_buckets:
                    for k in range(1, K+1):
                        u.append((i,r,d,k))
        self.x = self.model.addVars(u)

        # Define z(i) indicating the number of rooms prelim i is assigned to
        v = []
        for i in exams.index:
            v.append(i)
        self.z = self.model.addVars(v)
            
        
        return

    def add_constraints(self):
        '''
        Function to add all the constraints
        '''
        self.add_z_constraint()
        self.add_absolute_room_bound_constraint()
        self.add_unique_slot_constraint()
        self.add_room_use_constraint()
        self.add_x_const()
        self.add_enrollment_const()
        self.add_coenroll_conflict_const()

        self.model.update()

    def add_z_constraint(self):
        '''
        Add constraint to ensure z represents the number of classes a prelim is assigned to
        '''        
        for i in exams.index:
            u = quicksum(self.x[i,r,d,k] for r in room_buckets for d in exam_dates for k in range(1,K+1))
            self.model.addConstr(self.z[i] == u)
        return 

    def add_absolute_room_bound_constraint(self):
        '''
        Add constraint to ensure a single prelim is assigned to at most R rooms
        '''
        for i in exams.index:
            for r in range(1,self.R+1):
                self.model.addConstr(self.z[i] <= r)
        return 

    def add_unique_slot_constraint(self):
        '''
        Add constraint to ensure each prelim is assigned to a unique time slot
        '''
        for i in exams.index:
            a = quicksum(self.y[i,exams.prefdate[i],k] for k in range(1,K+1))
            b = quicksum(self.y[i,exams.prefdate2[i],k] for k in range(1,K+1))
            c = quicksum(self.y[i,exams.prefdate3[i],k] for k in range(1,K+1))
            self.model.addConstr(a+b+c == 1)
        return 

    def add_room_use_constraint(self):
        '''
        Add constraint to ensure each room_bucket is not over used
        '''
        for r in room_buckets:
            for d in exam_dates:
                for k in range(1,K+1):
                    u = quicksum(self.x[i,r,d,k] for i in exams.index)
                    self.model.addConstr(u <= r) 
        return 

    def add_x_const(self):
        '''
        Add constraint to ensure a room is only assigned during the correct time slot
        '''
        for i in exams.index:
            for d in exams.prefdate[i]:
                for k in range(1,K+1):
                    for r in room_buckets:
                        u = self.x[i,r,d,k]
                        v = self.R*self.y[i,d,k]
                        self.model.addConstr(u<=v)        
        return 

    def add_enrollment_const(self):
        '''
        Add constraint to ensure each exam is given enough seats
        ''' 
        for i in exams.index:
            rb = quicksum(room_buckets[i])
            u = quicksum(self.x[i,r,d,k] for r in room_buckets for d in exams.prefdate[i] for k in range(1,K+1))
            self.model.addConstr(rb*u >= room_buckets[i])        
        return 

    def add_coenroll_conflict_const(self):
        '''
        Add constraint to ensure that we try to schedule exams with coenrollment conflicts not at the same time 
        '''   
               
        bigC = coenroll_s21
        def C(u,v):
            if exams.prefdate[i] == (exams.prefdate[j] or exams.prefdate2[j] or exams.prefdate3[j]) and BigC > 1:
                return 1
            else:
                return 0
            
        u = (self.y[i,d,k] for i in exams.index for d in exams.prefdate[i] for k in range(1,K+1))
        v = (self.y[j,d,k] for j in exams.index for d in exams.prefdate[i] for k in range(1,K+1))
        self.model.addConstr(u+v-1 <= C(u,v))
                             
        return 

    def set_objective(self):
        '''
        Set the objective for the IP

        The objective has terms capturing
        (1) Weighted total number of rooms used 
        (2) Weighted number of classes that gor their second preference
        (3) Weighted number of classes that gor their third preference
        (4) number of coenrollment conflicts
        '''
        
        min(quicksum(self.rooms_used_weight*self.z[i] for i in exams.index)
           + quicksum(self.weight_pref_2*self.y[i,exams.prefdate2[i],k] for i in exams.index for k in range(1,K+1))
           + quicksum(self.weight_pref_3*self.y[i,xams.prefdate3[i],k] for i in exams.index for k in range(1,K+1))
           + quicksum(self.coenroll_s21*self.conenroll_s21 for i in self.conenroll_s21))
        
        self.model.update()
        return 

    def solve(self):
        '''
        Function to solve the IP problem 
        '''
        # solve the model
        self.model.optimize()
        return
   

## Solve

In [79]:
scheduler = PrelimExamScheduler(exams, exam_dates,
                                room_buckets,
                                coenroll_s21,
                                K = K, 
                                R = 10,
                                rooms_used_weight = 1,
                                weight_pref_2 = 1,
                                weight_pref_3 = 2
                                )

In [83]:
solution = scheduler.solve()

AttributeError: 'PrelimExamScheduler' object has no attribute 'model'